In [2]:
import numpy as np
import glob, os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import pandas as pd

from pathlib import Path
import librosa
import scipy

## Defined Functions:

In [3]:
def extract_features(signal, fs):
    # Always Need to give an input for sampling rate so that other higher level features can be extracter
   
    zcr = librosa.feature.zero_crossing_rate(signal); #Zero Crossing Rate
    spec_cent = librosa.feature.spectral_centroid(signal,fs);  #Spectral Centroid
    tontz = librosa.feature.tonnetz(signal,fs) #tonal centroid features
    S = librosa.feature.melspectrogram(signal,fs) # Mel Spectorgram
    mfc = librosa.feature.mfcc(signal,fs,S) #Mel frequency Coefficents
    return zcr, spec_cent, tontz, S, mfc
  
    
## Function to load all .wav files in a given folder to an array of (data, sample rate) ##
def load_folder(data_path):
    # input string of path to specific folder
    samples = [];
    
    for file in glob.glob(os.path.join(data_path,'*.wav')):
        temp,sr = librosa.load(file);
        temp = librosa.util.fix_length(temp,2*sr);
        samples.append([temp,sr])
        
    return samples
            

## Load Sample Files:

In [8]:
# CHANGE PATHS TO YOUR FOLDERS
path = 'nsynth\nsynth_bass_subset'; 
# snare_path = 'audio\snareSamples';

nsynthSamples = load_folder(path);
# snareSamples = load_folder(snare_path);
OtherSamples = load_folder('audio\kickSamples')

In [7]:
np.shape(nsynthSamples) ## If files loaded correctly should return (# samples, 2)

# bassSamp = bassSamples[:8394]
bassSamp = nsynthSamples[:1000]
# vocalSamp = bassSamples[8395:9395]



## Extract Features:

In [7]:
bassFeat = [extract_features(x[0], x[1]) for x in bassSamp]
vocalFeat = [extract_features(x[0], x[1]) for x in vocalSamp]



NameError: name 'bassSamp' is not defined

$\textbf{Features output as lists of feature attributes for each sample:}$

kickFeat[k][0] -> zcr features for kick sample k

kickFeat[k][1] -> spec centroid for kick sample k

kickFeat[k][2] -> tonal centroid for kick sample k

kickFeat[k][3] -> mel spec for kick smaple k

kickFeat[k][4] -> mfcs for kick sample k


In [11]:
# Organize extractions by feature classes rater than samples

# assign binary tags for sample types
class_tags = np.ones(len(bassSamp)) 
class_tags = np.append(class_tags, np.zeros(len(vocalSamp)))

zero_co= []
cent_freq = []
tonal_ctroid = []
mfccs = []

i = 0
for s in bassFeat:
    zero_co.append(s[0][i])
    cent_freq.append( s[1][i])
    tonal_ctroid.append(s[2][i])
    mfccs.append(s[4][i])


for s in vocalFeat:
    zero_co.append(s[0][i])
    cent_freq.append( s[1][i])
    tonal_ctroid.append(s[2][i])
    mfccs.append(s[4][i])

    
zero_co = (zero_co, class_tags)
cent_freq = (cent_freq,  class_tags)
tonal_ctroid = (tonal_ctroid, class_tags)
mfccs = (mfccs,  class_tags)

# Create dataset from extracted features

datasets = [zero_co,cent_freq,tonal_ctroid,mfccs]
h = .02  # step size in the mesh

## Classification:


In [12]:

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    # Classifiers are configured here
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]






## Perprocessing breakdown ##
i = 0 
analysis_data = np.zeros((len(datasets),len(names)))
for ds in datasets:
    #ds is a tuple
    # unpacks X as the feature training set, y as the target values
    X, y = ds
    # StandardScaler will remove the mean and scaling to unit variance
    # fit_transform: fits then transforms
    # fit: Compute the mean and std to be used for later scaling
    # transform: Perform standardization by centering and scaling
    X = StandardScaler().fit_transform(X)
    # splits the features into random train and test data
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)

    j=0
    for name, clf in zip(names, classifiers):
        clf.fit(X_train,y_train) #trains model
        score = clf.score(X_test, y_test) # tests model
        analysis_data[i][j] = score
        # clf may be the model, so it will need to be saved somewhere if it is accurate
        j+=1
    i+=1
        
df=pd.DataFrame(analysis_data,columns = names)
df.index= ['zero_crossing','centrial_frequency','tonnetz','mfccs']

## Display Analysis of Results:

In [13]:
df

,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA
zero_crossing,0.97000,0.92375,0.94000,0.98875,0.95000,0.88750,0.95625,0.97750,0.68500,0.89750
centrial_frequency,0.99500,0.91500,0.94125,1.00000,0.98000,0.96750,0.99125,0.99750,0.80625,0.96375
tonnetz,0.83125,0.74125,0.94875,0.97125,0.80250,0.83875,0.98750,0.88875,0.75750,0.95125
mfccs,0.99500,1.00000,0.97125,1.00000,0.99875,0.99875,1.00000,1.00000,0.95625,0.99500


In [9]:
#**FEEL FREE TO IGNORE*** (this was used for the presentation slides)

# # Reformatting Results Data 

# import pandas as pd
# import numpy

# data=numpy.array([[0.97, 0.99, 0.83, 0.99],[0.92,0.91, 0.74, 1.00],[0.94, 0.94,0.95, 0.97],[0.98,1.00,0.97,1.00],[0.95,0.98,0.80,0.99],[0.88,0.96,0.83,0.99],[0.95,0.99,0.98,1],[0.97,0.99,0.88,1.00],[0.68,0.80,0.75,0.95],[0.89,0.96,0.95,0.99]])
# names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes", "QDA"]
# tbl = pd.DataFrame(data.T,columns = names)
# tbl.index = ['zero_crossing','centrial_frequency','tonnetz','mfccs']

# tbl

,Nearest Neighbors,Linear SVM,RBF SVM,Gaussian Process,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA
zero_crossing,0.97,0.92,0.94,0.98,0.95,0.88,0.95,0.97,0.68,0.89
centrial_frequency,0.99,0.91,0.94,1.00,0.98,0.96,0.99,0.99,0.80,0.96
tonnetz,0.83,0.74,0.95,0.97,0.80,0.83,0.98,0.88,0.75,0.95
mfccs,0.99,1.00,0.97,1.00,0.99,0.99,1.00,1.00,0.95,0.99
